In [1]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surfs import surfs
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.cell import cell
from serpentGenerator.functions.cells import cells
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mix import mix
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.core import core
from serpentGenerator.functions.housing import housing
from serpentGenerator.functions.universe import universe
from serpentGenerator.functions.branches import branches

mat waterd600    -0.6 moder lwtr 1001   rgb 198 226 255
1001.06c	0.66667
8016.06c	0.33333


mat waterd700    -0.7 moder lwtr 1001   rgb 198 226 255
1001.06c	0.66667
8016.06c	0.33333


mat waterd800    -0.8 moder lwtr 1001   rgb 198 226 255
1001.06c	0.66667
8016.06c	0.33333


mix waterd600b0
waterd600	-1.0
boron	0.0


mix waterd600b750
waterd600	-0.99925
boron	-0.00075


mix waterd600b1500
waterd600	-0.9985
boron	-0.0015


mix waterd700b0
waterd700	-1.0
boron	0.0


mix waterd700b750
waterd700	-0.99925
boron	-0.00075


mix waterd700b1500
waterd700	-0.9985
boron	-0.0015


mix waterd800b0
waterd800	-1.0
boron	0.0


mix waterd800b750
waterd800	-0.99925
boron	-0.00075


mix waterd800b1500
waterd800	-0.9985
boron	-0.0015




In [2]:
fuel1 = MATLIB['UO2'].duplicateMat("fuel1")
fuel1.set('fractions', np.array([-0.04319337211, -0.83830401789, -.11850261])) # 4.9 % wt enrichment
fuel1.set('dens', -10.5216)
fuel1.set('rgb', "238 59 49")

fuel2 = fuel1.duplicateMat("fuel2")
fuel2.set('rgb', "255 165 0")

fuel3 = fuel1.duplicateMat("fuel3")
fuel3.set('rgb', "255 255 0")

fuel4 = fuel1.duplicateMat("fuel4")
fuel4.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261])) # 2.0 % wt enrichment 
fuel4.set('rgb', "255 255 255")

mod = MATLIB['lightWater'].duplicateMat("water")
# mod.set("isBurn", False)
mod.set('rgb', "198 226 255")

clad = MATLIB['Zr'].duplicateMat("clad")
# clad.set("isBurn", False)
clad.set('rgb', "183 183 183")

ref = MATLIB['plenum'].duplicateMat("reflector") # reflector plenum
# ref.set("isBurn", False)
ref.set('rgb', "204 153 255")

# b = MATLIB['boron'].duplicateMat("boron")
# b.set('rgb', "255 255 0")

ba = MATLIB['boron'].duplicateMat("B4C") # 90% wt enriched b-10

nb4c = 2.52*.60222/55.235
nb = 4*nb4c
nb10 = nb*.90
nb11 = nb*.10
nc12 = nb4c
b4cdens = -2.52

# ba.set("isBurn", False)
ba.set('rgb', "255 255 0")
ba.set('nuclides', np.array([5010, 5011, 6012]))
ba.set('fractions', np.array([0.105178192, 0.010628813, 0.028951751]))
ba.set('dens', b4cdens)
matList = mats()
matList.addMats([fuel1, fuel2, fuel3, fuel4,  mod, clad, ref, ba])

# print(matList.toString())

In [3]:
F1 = pin("F1", 3)
F1.set('materials', np.array([fuel1, clad, mod]))
F1.set('radii', np.array([.4095, .4750]))

F2 = F1.duplicatePin("F2")
F2.set('materials', np.array([fuel2, clad, mod]))

F3 = F1.duplicatePin("F3")
F3.set('materials', np.array([fuel3, clad, mod]))

F4 = F1.duplicatePin("F4")
F4.set('materials', np.array([fuel4, clad, mod]))


G1 = pin("G1", 3)
G1.set('materials', np.array([mod, clad, mod]))
G1.set('radii', np.array([.5715, .6120]))

W1 = pin("W1", 3)
W1.set('materials', np.array([mod]))

RT = pin("RT", 1) #top reflector
RT.set('materials', np.array([ref]))

RB =  RT.duplicatePin("RB")

pinList1 = pins()
pinList1.addPins([F1, F2, F3, F4, W1, RT, RB, G1])

#print(pinList.toString())

In [4]:
riClad = .57150
roClad = .61200

tClad = roClad - riClad

span1 = 70/4
gap1 = (360-span1*4)/4
theto1 = (90 - span1)/2
thetf1 = 360 - theto1

span2 = 55/4
gap2 = (360-span2*4)/4
theto2 = (90 - span2)/2
thetf2 = 360 - theto2

span3 = 60/4
gap3 = (360-span3*4)/4
theto3 = (90 - span3)/2
thetf3 = 360 - theto3

t1 = .090/10
t2 = .089/10
t3 = .019/10

riPad1 = riClad+ (tClad - t1)/2 
roPad1 = riPad1 + t1

riPad2 = riClad+ (tClad - t2)/2 
roPad2 = riPad2 + t2

riPad3 = riClad+ (tClad - t3)/2 
roPad3 = riPad3 + t3

vol1 = (span1/360)*np.pi*(roPad1*roPad1 - riPad1*riPad1)*200
print(vol1)

vol2 = (span2/360)*np.pi*(roPad2*roPad2 - riPad2*riPad2)*200
print(vol2)

vol3 = (span3/360)*np.pi*(roPad3*roPad3 - riPad3*riPad3)*200
print(vol3)

totvol = vol1*9 + vol2*12 + vol3*16
print(totvol)

r1 = [riClad, riPad1, roPad1, roClad]
r2 = [riClad, riPad2, roPad2, roClad]
r3 = [riClad, riPad3, roPad3, roClad]

crs1 = [None]*len(r1)
baps1 = [None]*len(r1)
bapscc1 = [None]*len(r1)
clps1 = [None]*len(r1)
clpscc1 = [None]*len(r1)

crs2= [None]*len(r2)
baps2= [None]*len(r2)
bapscc2 = [None]*len(r2)
clps2= [None]*len(r2)
clpscc2= [None]*len(r2)

crs3= [None]*len(r3)
baps3= [None]*len(r3)
bapscc3 = [None]*len(r3)
clps3= [None]*len(r3)
clpscc3= [None]*len(r3)


alpha11= theto1
alpha12 = alpha11 + span1
alpha13 = thetf1
alpha14 = (alpha13 + gap1) % 360
                    
alpha21 = theto2
alpha22 = alpha21 + span2
alpha23 = thetf2
alpha24 = (alpha23 + gap2) % 360

alpha31 = theto3
alpha32 = alpha31 + span3
alpha33 = thetf3
alpha34 = (alpha33 + gap3) % 360

for i in range(0, len(r1)):
    crs1[i] = surf("crt1"+str(i+1), "cyl", np.array([0.0, 0.0, r1[i]]))
    baps1[i] = surf("bat1"+str(i+1), "pad", np.array([0.0, 0.0, riPad1, roPad1, alpha11, alpha12]))
    clps1[i] = surf("clt1"+str(i+1), "pad", np.array([0.0, 0.0, riPad1, roPad1, alpha13, alpha14]))
    bapscc1[i] = cell("bapadt1"+str(i+1), np.array([baps1[i]]), np.array([1]), ba.id)
    clpscc1[i] = cell("clpadt1"+str(i+1), np.array([clps1[i]]), np.array([1]), clad.id)
    alpha11 = alpha12 + gap1
    alpha12 = alpha11 + span1
    alpha13 = alpha14 + span1
    alpha14 = alpha13 + gap1
    
for i in range(0, len(r2)):
    crs2[i] = surf("crt2"+str(i+1), "cyl", np.array([0.0, 0.0, r2[i]]))
    baps2[i] = surf("bat2"+str(i+1), "pad", np.array([0.0, 0.0, riPad2, roPad2, alpha21, alpha22]))
    clps2[i] = surf("clt2"+str(i+1), "pad", np.array([0.0, 0.0, riPad2, roPad2, alpha23, alpha24]))
    bapscc2[i] = cell("bapadt2"+str(i+1), np.array([baps2[i]]), np.array([1]), ba.id)
    clpscc2[i] = cell("clpadt2"+str(i+1), np.array([clps2[i]]), np.array([1]), clad.id)
    alpha21 = alpha22 + gap2
    alpha22 = alpha21 + span2
    alpha23 = alpha24 + span2
    alpha24 = alpha23 + gap2
    
for i in range(0, len(r3)):
    crs3[i] = surf("crt3"+str(i+1), "cyl", np.array([0.0, 0.0, r3[i]]))
    baps3[i] = surf("bat3"+str(i+1), "pad", np.array([0.0, 0.0, riPad3, roPad3, alpha31, alpha32]))
    clps3[i] = surf("clt3"+str(i+1), "pad", np.array([0.0, 0.0, riPad3, roPad3, alpha33, alpha34]))
    bapscc3[i] = cell("bapadt3"+str(i+1), np.array([baps3[i]]), np.array([1]), ba.id)
    clpscc3[i] = cell("clpadt3"+str(i+1), np.array([clps3[i]]), np.array([1]), clad.id)
    alpha31 = alpha32 + gap3
    alpha32 = alpha31 + span3
    alpha33 = alpha34 + span3
    alpha34 = alpha33 + gap3
    
cc11 = cell("cc11", np.array([crs1[0]]), np.array([1]), mod.id)
cc12 = cell("cc12", np.array([crs1[0], crs1[1]]), np.array([0, 1]), clad.id)
cc13 = cell("cc13", np.array([crs1[2], crs1[3]]), np.array([0, 1]), clad.id)
cc14 = cell("cc14", np.array([crs1[3]]), np.array([0]), mod.id)

cc21 = cell("cc21", np.array([crs2[0]]), np.array([1]), mod.id)
cc22 = cell("cc22", np.array([crs2[0], crs2[1]]), np.array([0, 1]), clad.id)
cc23 = cell("cc23", np.array([crs2[2], crs2[3]]), np.array([0, 1]), clad.id)
cc24 = cell("cc24", np.array([crs2[3]]), np.array([0]), mod.id)

cc31 = cell("cc31", np.array([crs3[0]]), np.array([1]), mod.id)
cc32 = cell("cc32", np.array([crs3[0], crs3[1]]), np.array([0, 1]), clad.id)
cc33 = cell("cc33", np.array([crs3[2], crs3[3]]), np.array([0, 1]), clad.id)
cc34 = cell("cc34", np.array([crs3[3]]), np.array([0]), mod.id)

surfList1 = surfs()
surfList1.addSurfs(crs1)
surfList1.addSurfs(baps1)
surfList1.addSurfs(clps1)

cellList1 = cells()
cellList1.addCells(bapscc1)
cellList1.addCells(clpscc1)
cellList1.addCells([cc11, cc12, cc13, cc14])

T1 = universe("T1")
T1.surfs = surfList1
T1.cells = cellList1

# print(bigT1.toString())

surfList2 = surfs()
surfList2.addSurfs(crs2)
surfList2.addSurfs(baps2)
surfList2.addSurfs(clps2)

cellList2 = cells()
cellList2.addCells(bapscc2)
cellList2.addCells(clpscc2)
cellList2.addCells([cc21, cc22, cc23, cc24])

T2 = universe("T2")
T2.surfs = surfList2
T2.cells = cellList2

# print(bigT2.toString())

surfList3 = surfs()
surfList3.addSurfs(crs3)
surfList3.addSurfs(baps3)
surfList3.addSurfs(clps3)

cellList3 = cells()
cellList3.addCells(bapscc3)
cellList3.addCells(clpscc3)
cellList3.addCells([cc31, cc32, cc33, cc34])

T3 = universe("T3")
T3.surfs = surfList3
T3.cells = cellList3

#print(bigT3.toString())

0.3253315542333074
0.2527774536463571
0.05886951933745477
6.903225741255328


In [5]:
FC1 = pinStack("FC1", 0, 0, 5)
FC1.setStack(np.array([RB, F4, F1, F4, RT]), np.array([0.0, 20.0, 25.0, 210.0, 220.0]))

FC2 = FC1.duplicateLat("FC2")
FC2.replacePin(F1, F2)

FC3 = FC1.duplicateLat("FC3")
FC3.replacePin(F1, F3)

GT1 = pinStack("GT1", 0, 0, 3)
GT1.setStack(np.array([RB, T1, RT]), np.array([0.0, 20.0, 220.0]))

GT2 = GT1.duplicateLat("GT2")
GT2.replacePin(T1, T2)

GT3 = GT1.duplicateLat("GT3")
GT3.replacePin(T1, T3)

WG1 = GT1.duplicateLat("WG1")
WG1.replacePin(T1, W1)

CT1 = pinStack("CT1", 0, 0, 3)
CT1.setStack(np.array([RB, G1, RT]), np.array([0.0, 20.0, 220.0]))
               
latList1 = lats()
latList1.addLats([FC1, FC2, FC3, GT1, GT2, GT3, CT1, WG1])


# print(GT1.toString())

In [6]:
FA1 = sqLat("FA1", 0, 0, 19, 1.260)

FA1Map = np.array([
    [WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, CT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1]
])

FA1.setMap(FA1Map)

FA2 = FA1.duplicateLat("FA2")
FA2.replacePin(FC1, FC2)
FA2.replacePin(GT1, GT2)

FA3 = FA1.duplicateLat("FA3")
FA3.replacePin(FC1, FC3)
FA3.replacePin(GT1, GT3)

WA1 = FA1.duplicateLat("WA1")
WA1.replacePin(FC1, WG1)
WA1.replacePin(GT1, WG1)



In [7]:
latList1.addLats([FA1, FA2, FA3, WA1])

# print(latList.toString())

In [ ]:
CORE1 = sqLat("CORE1", 0, 0, 11, 21.50364)

CORE1Map = np.array([
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, FA3, FA3, FA3, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, FA3, FA2, FA2, FA2, FA3, WA1, WA1, WA1],
[WA1, WA1, FA3, FA2, FA1, FA1, FA1, FA2, FA3, WA1, WA1],
[WA1, WA1, FA3, FA2, FA1, FA1, FA1, FA2, FA3, WA1, WA1],
[WA1, WA1, FA3, FA2, FA1, FA1, FA1, FA2, FA3, WA1, WA1],
[WA1, WA1, WA1, FA3, FA2, FA2, FA2, FA3, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, FA3, FA3, FA3, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1]
])

CORE1.setMap(CORE1Map)

In [ ]:
width = 21.50364*8
height  = 240
radii = [0, 5, 25, 45]
mats = [clad, mod, clad]

h1 = housing(width=width, height=height, defaultCRFlag=True, radiiCR= radii, matsCR= mats)

# print(h1.toString())

In [ ]:
input1 = core(mainUniv=CORE1, housing=h1, lats=latList1, pins=pinList1, materials=matList, flagBurn=True, flagBranch=False, flagXS = False)
input1.setXS(ngroups=2, ebounds=np.array([0.625E-6]), universes=[RB, RT], setFPPXS = True, setADF = False)
input1.setSettings(power=200E6, bc=[2, 2, 1], sym=0, egrid = [5e-5, 1e-9, 15.0],  nps=80000, nact=80, nskip=20, setPCC = False, 
                   misc = ["ene 1 1 1E-11 0.625E-6 15", "include bigT3D.txt.mvol"])
# input1.setPlot(types=[1, 3], res = 10000, plotOnly = False)

In [ ]:
inventory = np.array([541350, 922340, 922350, 942390])
burnPoints = np.array([
    0.10000,
    0.20000,
    0.40000,
    0.60000,
    0.80000,
    1.00000,
    1.50000,
    2.00000,
    4.00000,
    6.00000,
    8.00000,
   10.00000,
   12.00000,
   14.00000,
   16.50000,
   18.00000,
   20.00000,
   22.50000,
   25.00000,
   27.50000,
   30.00000,
   32.00000,
])
input1.setBurnup(inventory=inventory, burnPoints=burnPoints, isDayTot=False)

In [ ]:
absPath = "/mnt/c/Users/iaguirre6/Documents/xsdata/endfb7"
thermLib = "therm lwtr lwj3.11t"
input1.setXSLib(absPath, thermLib)

In [ ]:
input1.writeFile("bigT3D.txt")
text_file = open("bigT3D.txt") #Display contents of created inputFile
file_content = text_file.read()
print(file_content)
text_file.close()
